## PATH SETUP

In [1]:
# MUST be first cell: set multiprocessing method for Windows
import torch.multiprocessing as mp
try:
    mp.set_start_method("spawn", force=True)
except RuntimeError:
    pass  # Already set

In [2]:
import os
import sys
from pathlib import Path

# FOR LOCAL USE THIS LINES
current = Path.cwd()
src_path = current / "src" if (current / "src").exists() else current.parent

# FOR COLAB USE THIS LINE INSTEAD
# After -b insert the branch name if needed
# !git clone -b refactoring https://github.com/MatteoCamillo-code/GeoLoc-CVCS.git
# src_path = Path("/content/GeoLoc-CVCS/src").resolve()

sys.path.insert(0, str(src_path))

from utils.paths import find_project_root

# Set working directory and sys.path properly
project_root = find_project_root(src_path)
data_dir = project_root / "data"
os.chdir(project_root)
sys.path.insert(0, str(project_root / "src"))
print("CWD:", Path.cwd())

CWD: F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS


## IMPORT

In [12]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
from torch.optim.lr_scheduler import StepLR

from configs.baseline_multi_head import TrainConfig

from utils.seed import seed_everything
from utils.io import save_json
from training.runner import fit

from models.multi_head_classifier import MultiHeadClassifier


In [4]:
cfg = TrainConfig()
seed_everything(cfg.seed)

device = cfg.device if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [5]:
import kagglehub

path = kagglehub.dataset_download("josht000/osv-mini-129k")
path = path + "/osv5m"
print("Path to dataset files:", path)

image_root = path + "/train_images"


c:\Users\camil\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\camil\.cache\kagglehub\datasets\josht000\osv-mini-129k\versions\1/osv5m


In [6]:
train_val_path = data_dir / "metadata/s2-geo-cells/train_val_split_geocells.csv"
cell_centers_path = data_dir / "metadata/s2-geo-cells/cell_center_dataset.csv"

train_val_meta = pd.read_csv(train_val_path)
cell_centers_df = pd.read_csv(cell_centers_path)

print("Train/val CSV:", train_val_path)
print("Cell centers CSV:", cell_centers_path)


Train/val CSV: F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS\data\metadata\s2-geo-cells\train_val_split_geocells.csv
Cell centers CSV: F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS\data\metadata\s2-geo-cells\cell_center_dataset.csv


## DATALOADER

In [7]:
from dataset.dataloader_utils import create_dataloaders

IMG_SIZE = 224
TRAIN_SUBSET_PCT = 100.0  # Use 100% of training data (or set to e.g., 10.0 for 10%)
VAL_SUBSET_PCT = 100.0    # Use 100% of validation data

# Create all dataloaders with a single function call
loader_dict = create_dataloaders(
    image_root=image_root,
    csv_path=train_val_path,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    img_size=IMG_SIZE,
    seed=cfg.seed,
    train_subset_pct=TRAIN_SUBSET_PCT,
    val_subset_pct=VAL_SUBSET_PCT,
    augment=True,
    prefetch_factor=4,
    persistent_workers=True if cfg.num_workers > 0 else False,
    coarse_label_idx=cfg.coarse_label_idx,
)

train_loader = loader_dict["train_loader"]
val_loader = loader_dict["val_loader"]
label_maps = loader_dict["label_maps"]

print(f"Train size: {loader_dict['train_size']}")
print(f"Val size: {loader_dict['val_size']}")

Train size: 100863
Val size: 17803


In [8]:
weights = ResNet50_Weights.IMAGENET1K_V2
resnet = resnet50(weights=weights)

# number of classes depends on partition
num_classes = list(map(
    lambda idx: len(label_maps[f"label_config_{idx + 1}"]),
    cfg.coarse_label_idx
))

backbone = nn.Sequential(
    *list(resnet.children())[:-1],
    nn.Flatten(1)
)

FEAT_DIM = 2048  # resnet50 feature dimension

resnet = resnet.to(device)
# Optional: comment out if it causes issues on Windows/your PyTorch version
# model = torch.compile(model, backend="aot_eager")

model = MultiHeadClassifier(
    backbone=backbone,
    feat_dim=FEAT_DIM,
    head_dims=num_classes,
    dropout=cfg.dropout,
    coarse_level_idx=cfg.coarse_label_idx,
).to(device)

print("Output classes:", num_classes)


Output classes: [4741, 2508, 1336]


## MODEL

In [9]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)
# WITH SGD OPTIMIZER the convergence is very slow
# optimizer = torch.optim.SGD(
#     model.parameters(),
#     lr=cfg.lr,
#     momentum=cfg.momentum,
#     weight_decay=cfg.weight_decay,
#     nesterov=True
# )
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
scheduler = StepLR(optimizer, step_size=cfg.scheduler_step_size, gamma=cfg.scheduler_gamma)
scaler = torch.amp.GradScaler(device=cfg.device, enabled=cfg.amp)
torch.backends.cudnn.benchmark = True

In [10]:
history = fit(
    cfg=cfg,
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scaler=scaler,
    use_tqdm=cfg.use_tqdm,
    scheduler=scheduler
)

history

[15:54:11] INFO - Starting training...
c:\Users\camil\anaconda3\envs\tf\lib\site-packages\torch\optim\lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
[15:58:57] INFO - Epoch 1/30 | train loss=inf acc=11.97% | val loss=5.5420 acc=20.87% | time=285.88s
[16:01:59] INFO - Epoch 2/30 | train loss=4.7928 acc=32.45% | val loss=4.9843 acc=26.41% | time=181.68s
[16:05:10] INFO - Epoch 3/30 | train loss=4.0711 acc=39.05% | val loss=4.3701 acc=31.29% | time=191.10s
[16:08:54] INFO - Epoch 4/30 | train loss=3.3230 acc=48.57% | val loss=4.0006 acc=34.53% | t

{'train_loss': [inf,
  4.792789936065674,
  4.071115016937256,
  3.32296085357666,
  2.8081016540527344,
  2.42832612991333,
  2.1244301795959473,
  1.8834999799728394,
  1.686692714691162,
  1.5149797201156616,
  1.3729884624481201,
  1.2461293935775757,
  1.1420567035675049,
  1.047863245010376,
  0.9668921828269958,
  0.8928302526473999,
  0.8300926089286804,
  0.7729223966598511,
  0.7242932915687561,
  0.674541711807251,
  0.6357662081718445,
  0.6085418462753296,
  0.566426694393158,
  0.537732720375061,
  0.5140494704246521,
  0.487815260887146,
  0.4630505442619324,
  0.4451505243778229,
  0.4300491213798523,
  0.4076598882675171],
 'train_acc': [0.11965346336364746,
  0.32446539402008057,
  0.3904608488082886,
  0.48566100001335144,
  0.5584166646003723,
  0.6158762574195862,
  0.662397563457489,
  0.7005265355110168,
  0.7324962019920349,
  0.7612883448600769,
  0.7839301824569702,
  0.8050729036331177,
  0.8217409253120422,
  0.8368092775344849,
  0.8494839072227478,
  0.861

In [ ]:
save_json(
    obj=history,
    path=project_root / "outputs" / "history" / "baseline_multihead_history.json",
)